In [ ]:
# Importing necessary libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import csv
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, BicScore, BayesianEstimator, ExpectationMaximization, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork, BayesianModel
from pgmpy.inference import VariableElimination
import networkx as nx
from IPython.core.display import display, HTML
from pgmpy.metrics.metrics import correlation_score, log_likelihood_score, structure_score
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# Base URL with variable page number
urlBase = "https://polskiszekspir.uw.edu.pl/items/index/page/{}"

# Empty list for all topic page urls
urlTopics = []

# Function to generate URLs with increasing page numbers
def generate_page_urls(base_url, num_pages):
    for page_num in range(1, num_pages + 1):
        url = base_url.format(page_num)
        urlTopics.append(url)

# Generate URLs for the first 5 pages (you can adjust the number as needed)
generate_page_urls(urlBase, 5)

def extract_publications(soup):
    publications = []

    for publication in soup.find_all('div', class_='item--wrapper'):
        title_unfiltered = publication.find('span', class_='item__translator-title')
        if not title_unfiltered:
            print("Title not found")
            continue
        title_text = title_unfiltered.text.strip()

        # Extracting authors information
        authors = []
        for author_tag in publication.find_all('span', class_='item__translator-name'):
            author_text = author_tag.text.strip()
            # Extract only surname and name without dates
            author_name = re.match(r'([\w\-]+),\s*([\w\-]+)', author_text)
            if author_name:
                surname, name = author_name.groups()
                authors.append(f"{surname}, {name}")
            else:
                print(f"Could not extract author name from: {author_text}")

        # Convert authors list to a string
        authors_str = ', '.join(authors)

        # Extracting year information
        year_element = publication.find("div", class_="item__details--right").find("span", class_="item__details-place__name")
        year = year_element.text.strip() if year_element else None

        # Extracting place of publication
        place_element = publication.find("div", class_="item__details--left").find("span", class_="item__details-place__name")
        place = place_element.text.strip() if place_element else None

        # Extracting publisher
        publisher_element = publication.find_all("div", class_="item__details--left")[1].find("span", class_="item__details-place__name")
        publisher = publisher_element.text.strip() if publisher_element else None

        publications.append({
            "title": title_text,
            "author": authors_str,
            "year": year,
            "place of publication": place,
            "publisher": publisher
        })

    return publications




# Function to display the table with styling
def display_table_enhanced(publications):
    if not publications:
        print("No publications data found.")
        return
    df = pd.DataFrame(publications)
    df = df.drop_duplicates() # deleting duplicate records
    df = df.sort_values(by=['year', 'title'], ascending=[False, True]) # Sorting the entries in the table
    df.index = range(1, len(df) + 1)  # So the first position is no 1., not 0.

    # Improving visual aspects of the table with pandas styling
    styled_df = df.style.set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold')]},  # Bold headers
        {'selector': 'td', 'props': [('text-align', 'left')]},   # Aligning text to the left in text cells
        {'selector': 'th', 'props': [('text-align', 'left')]},   # Aligning text to the left in header cell
    ]).set_properties(**{'background-color': 'white', 'color': 'black'}) # Background colours
    return styled_df

# Collecting all publications from every single topic URL
all_publications = []
for topicUrl in urlTopics:
    print(f"Scraping: {topicUrl}")  # So it is displaying in the output which URL is being scraped right now
    try:
        response = requests.get(topicUrl)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            publications = extract_publications(soup)
            print(f"Found {len(publications)} publications.")
            all_publications.extend(publications)

            # Check if there is a next page link
            next_page_link = soup.find('a', class_='next')
            if next_page_link:
                next_page_num = int(next_page_link['href'].split('/')[-1])  # Extracting the page number from the URL
                next_page_url = urlBase.format(next_page_num)  # Generating the URL for the next page
                urlTopics.append(next_page_url)  # Add next page URL to the list for scraping
        else:
            print(f"Failed to retrieve {topicUrl}, status code {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {topicUrl}, error: {e}")
    time.sleep(1)  # So there is a polite delay between each of the requests

print(f"Total publications found: {len(all_publications)}")

styled_df = display_table_enhanced(all_publications)
styled_df # displaying the table


Scraping: https://polskiszekspir.uw.edu.pl/items/index/page/1
Found 12 publications.
Scraping: https://polskiszekspir.uw.edu.pl/items/index/page/2
Found 12 publications.
Scraping: https://polskiszekspir.uw.edu.pl/items/index/page/3
Found 12 publications.
Scraping: https://polskiszekspir.uw.edu.pl/items/index/page/4
Found 12 publications.
Scraping: https://polskiszekspir.uw.edu.pl/items/index/page/5
Found 12 publications.
Total publications found: 60


,title,author,year,place of publication,publisher
1,Burza,"Dzieduszycki, Wojciech",1904,Kraków,Drukarnia „Czasu”
2,Król Lear,"Dzieduszycki, Wojciech",1904,Kraków,Drukarnia „Czasu”
3,Romeo i Julia,"Dzieduszycki, Wojciech",1904,Kraków,Drukarnia „Czasu”
4,Miarka za miarkę,"Kasprowicz, Jan",1897,Lwów,Księgarnia Polska
5,Jak wam się podoba,"Lange, Antoni",1896,Lwów,Księgarnia Polska
6,Noc Trzech Króli,"Lange, Antoni",1896,Lwów,Księgarnia Polska
7,"Wszystko dobre, co się dobrze kończy","Porębowicz, Edward",1896,Lwów,Księgarnia Polska
8,Burza,"Paszkowski, Józef",1895,Lwów,Księgarnia Polska
9,Figle kobiet,"Paszkowski, Józef",1895,Lwów,Księgarnia Polska
10,Juljusz Cezar,"Pajgert, Adam",1895,Złoczów,Księgarnia Wilhelma Zukerkandla
